In [1]:
import time
import tarfile
import pandas as pd
import io


In [2]:
# Suppress warning for xlrd
import warnings
# Filter all warning that start with "WARNING"
warnings.filterwarnings('ignore')

In [3]:
file = '/Users/dirkocoetsee/Downloads/fuse-binaries-dec2014.tar.gz'
tar_stream = tarfile.open(file, mode='r|*')

vocab = {}
words_member = []
words = []
t0 = time.time()
total_time_open = 0
icounter = 0
total_counter = 0
df_list = []
for member_number, member in enumerate(tar_stream):
    t0_loop = time.time()
    if icounter > 1_000_001:
        break
    if not member.isfile():
        continue
    try:
        buffer = tar_stream.extractfile(member)
        dfs = pd.read_excel(io.BytesIO(buffer.read()), sheet_name=None, header=None, index_col=None, nrows=100)
    except Exception as e:
        continue
    for dfi, df in enumerate(dfs.values()):
        # Iterate over all cells
        vals = {str(v)[:20] for v in df.values[:100, :100].flatten()}
        # Update vocab
        vocab_size = len(vocab)
        # Add new words to vocab
        vocab.update({v: vocab_size + i + 1 for i, v in
                      enumerate(val[:20] if isinstance(val, str) else val for val in vals if val not in vocab)})
        # Map df to vocab - all ints
        df_mapped = df.map(lambda x: vocab[x] if x in vocab else 0)
        df_mapped['f'] = member.name
        df_mapped['s'] = dfi

        # Add to list
        df_list.append(df_mapped)

        icounter += 1

        # Accumulate sheets and write to parquet
        if len(df_list) >= 1000:
            dfc = pd.concat(df_list)
            dfc.columns = dfc.columns.astype(str)
            print(f'{icounter}', f'{total_counter}', '  ', len(dfs), len(vocab), dfc.shape)
            dfc.to_parquet(f'sdata/df_{total_counter:04d}.parquet', compression='zstd')
            df_list = []
            total_counter += 1


1000 0    3 119052
2000 1    3 223194
3000 2    1 316852
4000 3    1 388517
5000 4    1 452028
6000 5    55 547117
7000 6    12 622429
8000 7    1 679367
9000 8    1 747868
10000 9    3 804043
11000 10    2 860935
12000 11    3 919014
13000 12    3 967490
14000 13    5 1026174
15000 14    1 1084276
16000 15    1 1138401
17000 16    64 1205638
18000 17    15 1265087
19000 18    3 1316135
20000 19    1 1368159
21000 20    1 1434733
22000 21    82 1495680
23000 22    6 1547988
24000 23    16 1599898
25000 24    3 1648327
26000 25    1 1692387
27000 26    3 1733597
28000 27    2 1782338
29000 28    7 1826140
30000 29    1 1878213
31000 30    1 1927275
32000 31    1 1981649
33000 32    26 2051131
34000 33    2 2097713
35000 34    1 2140532
36000 35    14 2174450
37000 36    9 2211417
38000 37    1 2264908
39000 38    1 2310930
40000 39    1 2351135
41000 40    1 2399340
42000 41    1 2441652
43000 42    6 2482534
44000 43    17 2527992
45000 44    3 2562873
46000 45    1 2596777
47000 46   

In [4]:
# Save vocab
vocab_df = pd.DataFrame(vocab.items(), columns=['word', 'id'])
vocab_df.to_parquet('sdata/vocab.parquet', compression='zstd')

In [ ]:
!ls -lah sdata

In [38]:
!ls -lah sdata

total 2880
drwxr-xr-x@ 13 dirkocoetsee  staff   416B Feb 16 17:31 .
drwxr-xr-x@ 14 dirkocoetsee  staff   448B Feb 16 17:32 ..
-rw-r--r--@  1 dirkocoetsee  staff   681K Feb 16 17:32 df_0000.parquet
-rw-r--r--@  1 dirkocoetsee  staff   100K Feb 16 17:27 df_0001.parquet
-rw-r--r--@  1 dirkocoetsee  staff    63K Feb 16 17:27 df_0002.parquet
-rw-r--r--@  1 dirkocoetsee  staff    68K Feb 16 17:27 df_0003.parquet
-rw-r--r--@  1 dirkocoetsee  staff    72K Feb 16 17:27 df_0004.parquet
-rw-r--r--@  1 dirkocoetsee  staff    66K Feb 16 17:27 df_0005.parquet
-rw-r--r--@  1 dirkocoetsee  staff    57K Feb 16 17:27 df_0006.parquet
-rw-r--r--@  1 dirkocoetsee  staff    76K Feb 16 17:27 df_0007.parquet
-rw-r--r--@  1 dirkocoetsee  staff    76K Feb 16 17:27 df_0008.parquet
-rw-r--r--@  1 dirkocoetsee  staff    58K Feb 16 17:28 df_0009.parquet
-rw-r--r--@  1 dirkocoetsee  staff    20K Feb 16 17:31 vocab.parquet


In [5]:
import duckdb


In [6]:
res = duckdb.sql("select count(*) from 'sdata/*.parquet'")
res.show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│     21393957 │
└──────────────┘



In [10]:
res = duckdb.sql("""
    select distinct f, s from 'sdata/df*.parquet'
    limit 10
""")
res.show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────────────────────────────────────────┬───────┐
│                        f                         │   s   │
│                     varchar                      │ int64 │
├──────────────────────────────────────────────────┼───────┤
│ cc-binaries/52becece-39a8-4ca0-a694-eef0d90b908c │     0 │
│ cc-binaries/017abfa3-0818-4d93-9b25-c02a153956bb │     2 │
│ cc-binaries/8accbf39-c537-4c26-ab55-e629b8ea5300 │     0 │
│ cc-binaries/cf6358a0-90c6-4a49-b19d-b37cac6c86c3 │     0 │
│ cc-binaries/4e1a0951-75bc-42fc-8e83-df3a873f9cab │     0 │
│ cc-binaries/2320fcfd-f62f-4eb3-8d3e-2c77a826dba4 │     1 │
│ cc-binaries/c77d41b4-0f61-4efc-9821-e863e948e403 │     0 │
│ cc-binaries/a37384b8-b8c7-49b1-afce-b5c13bc414c4 │     0 │
│ cc-binaries/0278fe50-e5ba-49dc-8d8b-c3e7c8b0076d │     0 │
│ cc-binaries/2de92e95-ad38-4cdb-b62c-dd599158b2e1 │     0 │
├──────────────────────────────────────────────────┴───────┤
│ 10 rows                                        2 columns │
└───────────────────────

In [17]:
res = duckdb.sql("""
with sdata as (
    select f, s, "0", "1", "2", "3", "4", "5", "6", "7", "8", "9" from read_parquet('sdata/df*.parquet', union_by_name=True)
), top_sheet as (
    select distinct f, s from sdata
    limit 2
)
select * exclude f from sdata join top_sheet using (f, s)
limit 10
""")
res.show()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌───────┬────────┬──────────┬─────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬───────┐
│   s   │   0    │    1     │    2    │   3    │   4    │   5    │   6    │   7    │   8    │   9    │   s   │
│ int64 │ double │  double  │ double  │ double │ double │ double │ double │ double │ double │ double │ int64 │
├───────┼────────┼──────────┼─────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼───────┤
│     0 │ 1345.0 │ 317161.0 │ 30213.0 │   NULL │   NULL │   NULL │   NULL │   NULL │   NULL │   NULL │     0 │
│     0 │    0.0 │    184.0 │ 97038.0 │   NULL │   NULL │   NULL │   NULL │   NULL │   NULL │   NULL │     0 │
│     0 │    0.0 │    184.0 │ 30226.0 │   NULL │   NULL │   NULL │   NULL │   NULL │   NULL │   NULL │     0 │
│     0 │    0.0 │    184.0 │  2501.0 │   NULL │   NULL │   NULL │   NULL │   NULL │   NULL │   NULL │     0 │
│     0 │ 1305.0 │ 317168.0 │ 30210.0 │   NULL │   NULL │   NULL │   NULL │   NULL │   NULL │   NULL │     0 │
│